In [17]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import skimage.transform
import skimage.filters
import skimage.feature
import skimage.restoration
from skimage.transform import resize
from skimage.filters import sobel
from skimage.feature import canny
from skimage.restoration import denoise_bilateral, denoise_tv_chambolle
from skimage.io import imread, imshow
from sklearn.cluster import KMeans
from scipy import misc
from skimage.color import rgb2gray
from skimage.io import imread, imshow
import shutil
import glob

import boto3
import sys
sys.path.insert(0,'/home/newton/images')
import os
import pickle
import dill

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
# tf.enable_eager_execution()

import tensorflow_hub as hub
# import tensorflow_datasets as tfds

from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import model_from_json
import tensorflow.keras.experimental
import keras.applications.mobilenet
from keras.preprocessing import image
import file_image_processor

In [2]:
base_dir= '/home/newton/images/glasses_train'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')
batch_size = 100
IMG_W = 224
IMG_H = 224

In [3]:
image_gen_train = ImageDataGenerator(rescale=1/255)
train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=batch_size, 
                                                directory=train_dir, 
                                                shuffle=True, 
                                                target_size=(IMG_W,IMG_H),
                                                class_mode='sparse'
                                                )

Found 6400 images belonging to 2 classes.


In [4]:
image_gen_val = ImageDataGenerator(rescale=1/255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size, 
                                                 directory=val_dir, 
                                                 target_size=(IMG_W, IMG_H),
                                                 class_mode='sparse')

Found 1600 images belonging to 2 classes.


In [5]:
URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/2"

feature_extractor = hub.KerasLayer(URL)

feature_extractor.trainable = False

m2_glasses = tf.keras.Sequential([
  feature_extractor,
  layers.Dense(5, activation='softmax')
])

In [13]:
train_data_gen[0][0].shape

(100, 224, 224, 3)

In [6]:
m2_glasses.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
epochs = 5

history = m2_glasses.fit(
    train_data_gen,
    epochs=epochs,
    validation_data=val_data_gen
)

Epoch 1/5
64/64 [==============================] - 229s 4s/step - loss: 0.4791 - acc: 0.6399 - val_loss: 0.2753 - val_acc: 0.8756
Epoch 2/5
64/64 [==============================] - 226s 4s/step - loss: 0.2555 - acc: 0.8838 - val_loss: 0.2299 - val_acc: 0.9006
Epoch 3/5
64/64 [==============================] - 224s 3s/step - loss: 0.2211 - acc: 0.9088 - val_loss: 0.2146 - val_acc: 0.9137
Epoch 4/5
64/64 [==============================] - 223s 3s/step - loss: 0.1999 - acc: 0.9281 - val_loss: 0.1966 - val_acc: 0.9262
Epoch 5/5
64/64 [==============================] - 224s 4s/step - loss: 0.1879 - acc: 0.9275 - val_loss: 0.1863 - val_acc: 0.9281


In [8]:
m2_glasses.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [9]:
m2_glasses.save('./models/glasses_model_1.h5')

In [10]:
new_model = tf.keras.models.load_model('./models/glasses_model_1.h5',custom_objects={'KerasLayer':hub.KerasLayer})

In [11]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     multiple                  2257984   
_________________________________________________________________
dense (Dense)                multiple                  6405      
Total params: 2,264,389
Trainable params: 6,405
Non-trainable params: 2,257,984
_________________________________________________________________


In [ ]:
# prediction code below

In [25]:
im = prepare_image(os.path.join('/home/newton/images/gender_images/train/female','008642.jpg'))

In [26]:
result = new_model.predict(im)

In [ ]:
#  the results seem to be that the first number is glasses, the second is no glasses.
#  The other three are irrelevant.
#  the image used here has glasses.

In [27]:
print (result)

[[8.6931241e-01 1.3067637e-01 9.2824268e-07 1.0321990e-06 9.3097369e-06]]
